## Challenge: Exercise 4

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
%pylab inline

### To match searches with bookings

In [ ]:
import bz2
files_path = "/home/dsc/Repositories/Master-in-Data-Science/Data/"

##### Loading and formatting bookings

In [ ]:
bookings_file = bz2.BZ2File(files_path+"bookings.csv.bz2")
bookings_df = pd.read_csv(bookings_file, 
                          sep='^',
                          usecols=['dep_port','arr_port','brd_time           ','pax'], 
                          parse_dates=['brd_time           '])

In [ ]:
# Discarding blanks
bookings_df.dropna(inplace=True)

# Discarding "negative" bookings
bookings_df = bookings_df[bookings_df['pax'] >=1]
bookings_df.drop('pax', axis=1, inplace=True)

# Formatting bookings dataframe to match searches dataframe
bookings_df['Origin'] = bookings_df['dep_port'].apply(lambda s: s.strip())
bookings_df.drop('dep_port', axis=1, inplace=True)

bookings_df['Destination'] = bookings_df['arr_port'].apply(lambda s: s.strip())
bookings_df.drop('arr_port', axis=1, inplace=True)

bookings_df['Seg1Date'] = bookings_df['brd_time           '].apply(lambda d: pd.to_datetime(d.date()))
bookings_df.drop('brd_time           ', axis=1, inplace=True)

# Adding requested column "if search = booking"
bookings_df['Is_Booked'] = 1


##### Loading and formatting searches

In [ ]:
searches_file = bz2.BZ2File(files_path+"searches.csv.bz2")
searches_df = pd.read_csv(searches_file, 
                          sep='^',
                          nrows=1e5,
                          usecols=['Origin','Destination','Seg1Date'],
                          parse_dates=['Seg1Date'])
searches_df.reset_index(inplace=True)

##### Merging both dataframes

In [ ]:
# Obtaining searches with bookings
merged_df = pd.merge(searches_df,bookings_df,how="left").drop_duplicates()
merged_df['Is_Booked'].fillna(0,inplace=True)
merged_df.groupby('Is_Booked').count()
# merged_df.head()

In [ ]:
# Obtaining searches without bookings
merged0_df = pd.merge(searches_df,bookings_df,how="outer")
merged0_df['Is_Searched'].fillna(0, inplace=True)
merged0_df = merged0_df[merged0_df['Is_Searched'] == 0]

In [ ]:
# Appending and sorting final result
sol_df = merged1_df.append(merged0_df,ignore_index=True)
sol_df = sol_df.sort('index')
sol_df.head(10)

In [ ]:
sol_df.groupby('Is_Searched').count()